In [1]:
import pandas as pd
import datetime
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import catboost
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor


In [2]:

dataset = pd.read_csv('Data/Balitmore_salry.csv')


In [3]:
dataset.head()

Unnamed: 0                name                       jobtitle agencyid  \
0           0     Aaron,Keontae E                 AIDE BLUE CHIP   W02200   
1           1    Aaron,Patricia G  Facilities/Office Services II   A03031   
2           2       Aaron,Petra L     ASSISTANT STATE'S ATTORNEY   A29005   
3           3  Abaineh,Yohannes T                 EPIDEMIOLOGIST   A65026   
4           4    Abbene,Anthony M         POLICE OFFICER TRAINEE   A99416   

                    agency  hire_date  hire_month  hire_year  annualsalary  
0             Youth Summer         10           6       2013         11310  
1       OED-Employment Dev         24          10       1979         53428  
2  States Attorneys Office         25           9       2006         68300  
3   HLTH-Health Department         23           7       2009         62000  
4        Police Department         24           7       2013         43999

In [4]:
dataset.duplicated().sum()


0

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18895 entries, 0 to 18894
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    18895 non-null  int64 
 1   name          18895 non-null  object
 2   jobtitle      18895 non-null  object
 3   agencyid      18895 non-null  object
 4   agency        18895 non-null  object
 5   hire_date     18895 non-null  int64 
 6   hire_month    18895 non-null  int64 
 7   hire_year     18895 non-null  int64 
 8   annualsalary  18895 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 1.3+ MB


In [6]:
dataset.isnull().sum(axis = 0)

Unnamed: 0      0
name            0
jobtitle        0
agencyid        0
agency          0
hire_date       0
hire_month      0
hire_year       0
annualsalary    0
dtype: int64

In [7]:
dataset.head()


Unnamed: 0                name                       jobtitle agencyid  \
0           0     Aaron,Keontae E                 AIDE BLUE CHIP   W02200   
1           1    Aaron,Patricia G  Facilities/Office Services II   A03031   
2           2       Aaron,Petra L     ASSISTANT STATE'S ATTORNEY   A29005   
3           3  Abaineh,Yohannes T                 EPIDEMIOLOGIST   A65026   
4           4    Abbene,Anthony M         POLICE OFFICER TRAINEE   A99416   

                    agency  hire_date  hire_month  hire_year  annualsalary  
0             Youth Summer         10           6       2013         11310  
1       OED-Employment Dev         24          10       1979         53428  
2  States Attorneys Office         25           9       2006         68300  
3   HLTH-Health Department         23           7       2009         62000  
4        Police Department         24           7       2013         43999

In [8]:
dataset.describe(include=['object'])


name        jobtitle agencyid        agency
count              18895           18895    18895         18895
unique             18702            1068     1177            64
top     Stafford,Linda M  AIDE BLUE CHIP   P04001  Youth Summer
freq                   2            4817      316          4817

In [9]:
print(dataset['agencyid'].unique())


['W02200' 'A03031' 'A29005' ... 'W02674' 'W02382' 'W02339']


In [10]:
mask = dataset['agency'].str.contains('Health')

# replace all matching rows with the new value
dataset.loc[mask, 'agency'] = 'health_dept'

print(dataset['agency'].unique())

['Youth Summer' 'OED-Employment Dev' 'States Attorneys Office'
 'health_dept' 'Police Department' 'M-R Info Technology' 'Fire Department'
 "Sheriff's Office" 'Housing & Community Dev' 'Enoch Pratt Free Library'
 'General Services' 'HR-Test Monitor' 'TRANS-Traffic' 'TRANS-Highways'
 'R&P-Recreation (part-ti' 'DPW-Solid Waste (wkly)' 'Circuit Court'
 'DPW-Water & Waste Water' 'FIN-Collections' 'M-R Convention Center'
 'TRANS-Crossing Guards' 'Mayors Office' 'M-R Human Services'
 'FIN-Purchasing' 'COMP-Communication Ser' 'COMP-Audits' 'DPW-Solid Waste'
 'TRANS-Highways (wkly)' 'FIN-Risk Management Oper'
 'Municipal & Zoning Appeal' 'Legislative Reference' 'R&P-Parks (wkly)'
 'HR-Human Resources' 'R&P-Parks' "Orphan's Court" 'TRANS-Towing'
 'COMP-Real Estate' 'R&P-Administration' 'DPW-Administration'
 'Youth Temp Adult' 'City Council' 'Law Department' 'FPR Admin'
 'R&P-Recreation' 'Elections' 'FIN-Acct & Payroll' 'FIN-Admin & Budgets'
 'Planning Department' "COMP-Comptroller's O" 'HLTH-Hea

In [11]:
mask = dataset['agency'].str.contains('Mayor')

# replace all rows with the new value
dataset.loc[mask, 'agency'] = 'Mayor_office'

print(dataset['agency'].unique())

['Youth Summer' 'OED-Employment Dev' 'States Attorneys Office'
 'health_dept' 'Police Department' 'M-R Info Technology' 'Fire Department'
 "Sheriff's Office" 'Housing & Community Dev' 'Enoch Pratt Free Library'
 'General Services' 'HR-Test Monitor' 'TRANS-Traffic' 'TRANS-Highways'
 'R&P-Recreation (part-ti' 'DPW-Solid Waste (wkly)' 'Circuit Court'
 'DPW-Water & Waste Water' 'FIN-Collections' 'M-R Convention Center'
 'TRANS-Crossing Guards' 'Mayor_office' 'M-R Human Services'
 'FIN-Purchasing' 'COMP-Communication Ser' 'COMP-Audits' 'DPW-Solid Waste'
 'TRANS-Highways (wkly)' 'FIN-Risk Management Oper'
 'Municipal & Zoning Appeal' 'Legislative Reference' 'R&P-Parks (wkly)'
 'HR-Human Resources' 'R&P-Parks' "Orphan's Court" 'TRANS-Towing'
 'COMP-Real Estate' 'R&P-Administration' 'DPW-Administration'
 'Youth Temp Adult' 'City Council' 'Law Department' 'FPR Admin'
 'R&P-Recreation' 'Elections' 'FIN-Acct & Payroll' 'FIN-Admin & Budgets'
 'Planning Department' "COMP-Comptroller's O" 'HLTH-Heat

In [12]:
pd.set_option('display.max_rows', None)


In [13]:
print(dataset['agencyid'].unique())


['W02200' 'A03031' 'A29005' ... 'W02674' 'W02382' 'W02339']


In [14]:
print(dataset['jobtitle'].unique())


['AIDE BLUE CHIP' 'Facilities/Office Services II'
 "ASSISTANT STATE'S ATTORNEY" ... 'LACTATION TECHNICIAN'
 'PRESIDENT CITY COUNCIL' 'Research Analyst II']


In [15]:
dataset = dataset.drop('Unnamed: 0', axis=1)

In [16]:
dataset = dataset.drop('name', axis=1)

In [17]:
dataset['hire_year'] = dataset['hire_year'].astype(int)
dataset['hire_month'] = dataset['hire_month'].astype(int)
dataset['hire_date'] = dataset['hire_date'].astype(int)

dataset['hire_datetime'] = pd.to_datetime({
    'year': dataset['hire_year'],
    'month': dataset['hire_month'],
    'day': dataset['hire_date']
})

reference_date = datetime.datetime.now()

dataset['total_working_days'] = dataset['hire_datetime'].apply(
    lambda x: np.busday_count(x.date(), reference_date.date())
)

dataset = dataset.drop(['hire_datetime', 'hire_month', 'hire_date'], axis=1)

print(dataset.head())

                        jobtitle agencyid                   agency  hire_year  \
0                 AIDE BLUE CHIP   W02200             Youth Summer       2013   
1  Facilities/Office Services II   A03031       OED-Employment Dev       1979   
2     ASSISTANT STATE'S ATTORNEY   A29005  States Attorneys Office       2006   
3                 EPIDEMIOLOGIST   A65026              health_dept       2009   
4         POLICE OFFICER TRAINEE   A99416        Police Department       2013   

   annualsalary  total_working_days  
0         11310                2770  
1         53428               11543  
2         68300                4520  
3         62000                3782  
4         43999                2738  


In [18]:
dataset.describe()

hire_year   annualsalary  total_working_days
count  18895.000000   18895.000000        18895.000000
mean    2004.078328   39547.804763         5085.045197
std       10.391949   24865.587963         2704.046278
min     1952.000000       0.000000         2495.000000
25%     1998.000000   11310.000000         2809.000000
50%     2007.000000   36203.000000         4239.000000
75%     2013.000000   60300.000000         6582.000000
max     2014.000000  148200.000000        18675.000000

In [19]:
dataset.isnull().sum(axis = 0)

jobtitle              0
agencyid              0
agency                0
hire_year             0
annualsalary          0
total_working_days    0
dtype: int64

In [20]:
encoded_data = pd.get_dummies(dataset, columns=['jobtitle', 'agencyid', 'agency'])


In [21]:
encoded_data.head()

hire_year  annualsalary  total_working_days  jobtitle_911 LEAD OPERATOR  \
0       2013         11310                2770                       False   
1       1979         53428               11543                       False   
2       2006         68300                4520                       False   
3       2009         62000                3782                       False   
4       2013         43999                2738                       False   

   jobtitle_911 OPERATOR  jobtitle_911 OPERATOR SUPERVISOR  \
0                  False                             False   
1                  False                             False   
2                  False                             False   
3                  False                             False   
4                  False                             False   

   jobtitle_ACCOUNT EXECUTIVE  jobtitle_ACCOUNTANT I  jobtitle_ACCOUNTANT II  \
0                       False                  False                   False   
1                       False                  False                   False   
2                       False                  False                   False   
3                       False                  False                   False   
4                       False                  False                   False   

   jobtitle_ACCOUNTANT SUPV  ...  agency_States Attorneys Office  \
0                     False  ...                           False   
1                     False  ...                           False   
2                     False  ...                            True   
3                     False  ...                           False   
4                     False  ...                           False   

   agency_TRANS-Cross Guard-S  agency_TRANS-Crossing Guards  \
0                       False                         False   
1                       False                         False   
2                       False                         False   
3                       False                         False   
4                       False                         False   

   agency_TRANS-Highways  agency_TRANS-Highways (wkly)  agency_TRANS-Towing  \
0                  False                         False                False   
1                  False                         False                False   
2                  False                         False                False   
3                  False                         False                False   
4                  False                         False                False   

   agency_TRANS-Traffic  agency_Youth Summer  agency_Youth Temp Adult  \
0                 False                 True                    False   
1                 False                False                    False   
2                 False                False                    False   
3                 False                False                    False   
4                 False                False                    False   

   agency_health_dept  
0               False  
1               False  
2               False  
3                True  
4               False  

[5 rows x 2309 columns]

In [22]:
dataset.describe(include=['object'])


jobtitle agencyid        agency
count            18895    18895         18895
unique            1068     1177            61
top     AIDE BLUE CHIP   P04001  Youth Summer
freq              4817      316          4817

In [23]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [24]:
columns_list = encoded_data.columns.tolist()

print(columns_list)

['hire_year', 'annualsalary', 'total_working_days', 'jobtitle_911 LEAD OPERATOR', 'jobtitle_911 OPERATOR', 'jobtitle_911 OPERATOR SUPERVISOR', 'jobtitle_ACCOUNT EXECUTIVE', 'jobtitle_ACCOUNTANT I', 'jobtitle_ACCOUNTANT II', 'jobtitle_ACCOUNTANT SUPV', 'jobtitle_ACCOUNTANT TRAINEE', 'jobtitle_ACCOUNTING ASSISTANT II LIBRAR', 'jobtitle_ACCOUNTING ASST I', 'jobtitle_ACCOUNTING ASST II', 'jobtitle_ACCOUNTING ASST III', 'jobtitle_ACCOUNTING MANAGER', 'jobtitle_ACCOUNTING OPERATIONS OFFICER', 'jobtitle_ACCOUNTING SYSTEMS ADMINISTRAT', 'jobtitle_ACCOUNTING SYSTEMS ANALYST', 'jobtitle_ACCOUNTING SYSTEMS ANALYST I', 'jobtitle_ACCOUNTING SYSTEMS ANALYST II', 'jobtitle_ADM ASST', 'jobtitle_ADM COORDINATOR', 'jobtitle_ADMINISTRATIVE AIDE', 'jobtitle_ADMINISTRATIVE ANALYST I', 'jobtitle_ADMINISTRATIVE ANALYST II', 'jobtitle_ADMINISTRATIVE ASSISTANT', 'jobtitle_ADMINISTRATIVE COORDINATOR', 'jobtitle_ADMINISTRATIVE OFFICER', 'jobtitle_ADMINISTRATIVE OFFICER I', 'jobtitle_ADMINISTRATIVE OFFICER II', '

In [25]:
import re
encoded_data = encoded_data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [26]:
from collections import Counter

# Assuming columns_list is your list
num_duplicates = sum(count for count in Counter(columns_list).values() if count > 1)

print(f"Number of duplicated values: {num_duplicates}")


Number of duplicated values: 0


In [27]:
counts = Counter(columns_list)

# Extract duplicated values
duplicated_values = [item for item, count in counts.items() if count > 1]

print("Duplicated values:", duplicated_values)

Duplicated values: []


In [28]:
# Separate the features and the target variable
X = encoded_data.drop('annualsalary', axis=1)  # Ensure 'price' is the name of your target column
y = encoded_data['annualsalary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)

# Perform cross-validation
cv_scores = cross_val_score(xgb_reg, X, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean and standard deviation of the cross-validation scores
xbg_cv_scores_mean = -cv_scores.mean()
xgb_cv_scores_std = cv_scores.std()

print(f"CV Mean Squared Error: {xbg_cv_scores_mean}")
print(f"CV Standard Deviation: {xgb_cv_scores_std}")

CV Mean Squared Error: 104883937.10608396
CV Standard Deviation: 7837707.020652096


In [30]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)

# Fit the regressor to the training data
xgb_reg.fit(X_train, y_train)

y_train_pred = xgb_reg.predict(X_train)
y_test_pred = xgb_reg.predict(X_test)

# Compute the mean squared error of the predictions
mse_train = mean_squared_error(y_train, y_train_pred)
xgb_mse_test = mean_squared_error(y_test, y_test_pred)

print(f"Train Mean Squared Error: {mse_train}")
print(f"Test Mean Squared Error: {xgb_mse_test}")

Train Mean Squared Error: 101738634.41534527
Test Mean Squared Error: 98124570.65578356


In [31]:
lgb_reg = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=3)

# Perform cross-validation
cv_scores = cross_val_score(lgb_reg, X, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean and standard deviation of the cross-validation scores
lgb_cv_scores_mean = -cv_scores.mean()
lgb_cv_scores_std = cv_scores.std()

print(f"CV Mean Squared Error: {lgb_cv_scores_mean}")
print(f"CV Standard Deviation: {lgb_cv_scores_std}")

C:\ProgramData\anaconda3\envs\DataPreprocessingEnvironment\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\ProgramData\anaconda3\envs\DataPreprocessingEnvironment\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1035
[LightGBM] [Info] Number of data points in the train set: 15116, number of used features: 366
[LightGBM] [Info] Start training from score 39911.632839
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [32]:
lgb_reg = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=3)

# Fit the regressor to the training data
lgb_reg.fit(X_train, y_train)

y_train_pred = lgb_reg.predict(X_train)
y_test_pred = lgb_reg.predict(X_test)

# Compute the mean squared error of the predictions
mse_train = mean_squared_error(y_train, y_train_pred)
lgb_mse_test = mean_squared_error(y_test, y_test_pred)

print(f"Train Mean Squared Error: {mse_train}")
print(f"Test Mean Squared Error: {lgb_mse_test}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 15116, number of used features: 380
[LightGBM] [Info] Start training from score 39578.391572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [33]:
cat_reg = CatBoostRegressor(objective='RMSE', n_estimators=100, learning_rate=0.1, depth=3, verbose=False)

# Perform cross-validation
cv_scores = cross_val_score(cat_reg, X, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean and standard deviation of the cross-validation scores
cat_cv_scores_mean = -cv_scores.mean()
cat_cv_scores_std = cv_scores.std()

print(f"CV Mean Squared Error: {cat_cv_scores_mean}")
print(f"CV Standard Deviation: {cat_cv_scores_std}")

CV Mean Squared Error: 115391759.60417327
CV Standard Deviation: 7705037.891322094


In [34]:
cat_reg.fit(X_train, y_train)

# Predict on the training set and the test set
y_train_pred = cat_reg.predict(X_train)
y_test_pred = cat_reg.predict(X_test)

# Compute the mean squared error of the predictions
mse_train = mean_squared_error(y_train, y_train_pred)
cat_mse_test = mean_squared_error(y_test, y_test_pred)

print(f"Train Mean Squared Error: {mse_train}")
print(f"Test Mean Squared Error: {cat_mse_test}")


Train Mean Squared Error: 112965314.69277547
Test Mean Squared Error: 107297709.57102703


In [38]:

print("Cross-validation MSE:")
print(f"XGBoost: {xbg_cv_scores_mean}")
print(f"CatBoost: {lgb_cv_scores_mean}")
print(f"LightGBM: {cat_cv_scores_mean}")

print("\nTest MSE:")
print(f"XGBoost: {xgb_mse_test}")
print(f"CatBoost: {cat_mse_test}")
print(f"LightGBM: {lgb_mse_test}")

# Which model has the lowest cross-validation and test MSE
best_cv_model = min([('XGBoost', xbg_cv_scores_mean), ('CatBoost', lgb_cv_scores_mean), ('LightGBM', cat_cv_scores_mean)], key=lambda x: x[1])
best_test_model = min([('XGBoost', xgb_mse_test), ('CatBoost', cat_mse_test), ('LightGBM', lgb_mse_test)], key=lambda x: x[1])

print("\nBest performing model based on CV MSE:", best_cv_model[0])
print("Best performing model based on Test MSE:", best_test_model[0])


Cross-validation MSE:
XGBoost: 104883937.10608396
CatBoost: 106825277.93538801
LightGBM: 115391759.60417327

Test MSE:
XGBoost: 98124570.65578356
CatBoost: 107297709.57102703
LightGBM: 99391356.85216674

Best performing model based on CV MSE: XGBoost
Best performing model based on Test MSE: XGBoost
